# ⭐ Tutorial: Cross-Validation with PurgedKFold

This notebook demonstrates how to use the `PurgedKFold` cross-validator from the `RiskLabAI` library to run a robust financial backtest. 

Standard `KFold` cross-validation fails in finance because:
1.  **Leakage:** The training set can contain information that overlaps with the test set (e.g., from overlapping triple-barrier labels).
2.  **Embargo:** Information from the test set can 'leak' back into the training set that follows it.

`PurgedKFold` (De Prado, Chapter 7) solves this by:
1.  **Purging:** Removing any training samples whose labels are concurrent with test set samples.
2.  **Embargoing:** Removing training samples that immediately *follow* the test set to prevent look-ahead bias.

We will:
1.  Load data and generate features/labels.
2.  Define the `event_times` (`t1`) needed for purging.
3.  Instantiate and visualize `PurgedKFold`.
4.  Run a full backtest loop with a `RandomForestClassifier`.

## 0. Setup and Imports

In [ ]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

# Third-party for data and modeling
!pip install yfinance ta
import yfinance as yf
import ta
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# RiskLabAI Imports
from RiskLabAI.backtest.validation import PurgedKFold
import RiskLabAI.utils.publication_plots as pub_plots

# Setup plotting and configuration
pub_plots.setup_publication_style()
warnings.filterwarnings('ignore')

## 1. Load Data & Generate Signals

We will load 'SPY' data and generate some simple features and labels. For this example, we will assume a simple fixed-time horizon label (e.g., 20 days), so our `event_times` (`t1`) will be 20 days after the observation.

In [ ]:
data = yf.download('SPY', start='2010-01-01', end='2023-01-01')['Adj Close']
data = pd.DataFrame(data)

# --- 1. Generate Features (X) ---
X = pd.DataFrame(index=data.index)
X['rsi'] = ta.momentum.RSIIndicator(data['Adj Close'], window=14).rsi()
X['roc'] = ta.momentum.ROCIndicator(data['Adj Close'], window=10).roc()

# --- 2. Generate Labels (y) ---
# For simplicity, our label is: did the price go up 20 days from now?
look_forward = 20
y = data['Adj Close'].pct_change(look_forward).shift(-look_forward)
y = pd.Series((y > 0).astype(int), name='label')

# --- 3. Define Event Times (t1) ---
# This is CRITICAL for purging. It tells the validator when a label 'ends'.
# For our fixed horizon label, it's just the index + 20 days.
event_times = pd.Series(y.index, index=y.index).apply(lambda x: x + pd.DateOffset(days=look_forward))

# --- 4. Align Data ---
# Combine all data and drop NaNs to ensure X, y, and t1 are aligned
all_data = pd.concat([X, y, event_times.rename('t1')], axis=1).dropna()

X = all_data.drop(columns=['label', 't1'])
y = all_data['label']
event_times = all_data['t1']

print(f"Aligned data shapes: X={X.shape}, y={y.shape}, t1={event_times.shape}")
X.head()

## 2. Visualizing Purged K-Fold Splits

Before we run the backtest, let's visualize how `PurgedKFold` creates its splits. We'll use a `n_splits=10`.

Notice how the **blue (train)** bar is 'purged' (stops early) before the **orange (test)** bar begins, and an 'embargo' (a gap) is left after the test bar ends. This prevents any information leakage.

In [ ]:
n_splits = 10
embargo_pct = 0.01 # 1% embargo

cv = PurgedKFold(
    n_splits=n_splits,
    embargo_pct=embargo_pct
)

# --- Plotting the CV splits --- 
fig, ax = plt.subplots(figsize=(10, 5))
fold_indices = range(n_splits)

for i, (train_idx, test_idx) in enumerate(cv.split(X, y, event_times)):
    # Plot training indices
    ax.fill_betweenx(
        [i-0.4, i+0.4], 
        train_idx.min(), 
        train_idx.max(), 
        color='#00aedb', 
        alpha=0.7, 
        label='Train' if i == 0 else "_nolegend_"
    )
    
    # Plot testing indices
    ax.fill_betweenx(
        [i-0.4, i+0.4], 
        test_idx.min(), 
        test_idx.max(), 
        color='#d11141', 
        alpha=0.9, 
        label='Test' if i == 0 else "_nolegend_"
    )

ax.set_yticks(fold_indices)
ax.set_yticklabels([f'Fold {i+1}' for i in fold_indices])
ax.set_xlabel('Index')
ax.set_title(f'Purged K-Fold (n_splits={n_splits}, embargo={embargo_pct})')
ax.legend(loc='upper left')
plt.tight_layout()
plt.show()

## 3. Backtesting with PurgedKFold

This is the core of the tutorial. We loop through each (train, test) split generated by `cv.split()`:
1.  Train a `RandomForestClassifier` on the **training data**.
2.  Generate predictions on the **testing data**.
3.  Store these out-of-sample (OOS) predictions.

After the loop, we will have a complete set of OOS predictions for our entire dataset, which we can then score.

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
oos_predictions = pd.Series(index=y.index, dtype=float)

print("Running backtest...")
for fold, (train_idx, test_idx) in enumerate(cv.split(X, y, event_times)):
    print(f"--- Fold {fold+1}/{n_splits} ---")
    
    # Get data for this fold
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_test = X.iloc[test_idx]
    
    # 1. Train the model
    print(f"Training on {len(X_train)} samples...")
    model.fit(X_train, y_train)
    
    # 2. Generate predictions
    print(f"Predicting on {len(X_test)} samples...")
    y_pred = model.predict(X_test)
    
    # 3. Store OOS predictions
    oos_predictions.iloc[test_idx] = y_pred

print("\nBacktest complete.")

## 4. Evaluate OOS Accuracy

Now we can compare our complete set of OOS predictions against the true labels.

In [ ]:
# Drop NaNs from predictions (for any indices that were not in a test set)
oos_predictions = oos_predictions.dropna()
y_true = y.loc[oos_predictions.index]

accuracy = accuracy_score(y_true, oos_predictions)

print(f"Final Out-of-Sample Accuracy: {accuracy * 100:.2f}%")

**Conclusion:** We have successfully used `PurgedKFold` to run a robust cross-validation backtest, ensuring that no data leakage occurred between our training and testing sets. This OOS accuracy score is a reliable estimate of the model's performance.